In [1]:
import pandas as pd
import pandas as pd
import numpy as np




In [2]:
data = pd.read_csv('cleaned_data.csv')
data.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,6,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,0,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34,0,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,52,7,6,6,1,23,49,1,29,0,...,0,0,1,0,1,0,0,0,0,0
4,46,5,11,6,0,0,0,1,20,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X=data.drop(['Biopsy'], axis=1)
y=data['Biopsy']

Over Sampling

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np
import xgboost as xgb  # Import XGBoost

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote = []
precision_list_smote = []
recall_list_smote = []
f1_list_smote = []
entropy_list_smote = []
confusion_matrices_smote = []

accuracy_list_adasyn = []
precision_list_adasyn = []
recall_list_adasyn = []
f1_list_adasyn = []
entropy_list_adasyn = []
confusion_matrices_adasyn = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model without SMOTE/ADASYN
    xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE/ADASYN
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE/ADASYN
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE/ADASYN
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with SMOTE
    xgb_model_smote = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model_smote.fit(X_train_smote, y_train_smote)
    
    # Predictions and probabilities after SMOTE
    y_pred_smote = xgb_model_smote.predict(X_test)
    y_prob_smote = xgb_model_smote.predict_proba(X_test)
    
    # Calculate entropy after SMOTE
    entropy_smote = calculate_entropy(y_prob_smote)
    entropy_list_smote.append(entropy_smote)
    
    # Store metrics after SMOTE
    accuracy_list_smote.append(accuracy_score(y_test, y_pred_smote))
    precision_list_smote.append(precision_score(y_test, y_pred_smote))
    recall_list_smote.append(recall_score(y_test, y_pred_smote))
    f1_list_smote.append(f1_score(y_test, y_pred_smote))
    confusion_matrices_smote.append(confusion_matrix(y_test, y_pred_smote))
    
    # Apply ADASYN to the training data
    adasyn = ADASYN(random_state=42)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with ADASYN
    xgb_model_adasyn = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model_adasyn.fit(X_train_adasyn, y_train_adasyn)
    
    # Predictions and probabilities after ADASYN
    y_pred_adasyn = xgb_model_adasyn.predict(X_test)
    y_prob_adasyn = xgb_model_adasyn.predict_proba(X_test)
    
    # Calculate entropy after ADASYN
    entropy_adasyn = calculate_entropy(y_prob_adasyn)
    entropy_list_adasyn.append(entropy_adasyn)
    
    # Store metrics after ADASYN
    accuracy_list_adasyn.append(accuracy_score(y_test, y_pred_adasyn))
    precision_list_adasyn.append(precision_score(y_test, y_pred_adasyn))
    recall_list_adasyn.append(recall_score(y_test, y_pred_adasyn))
    f1_list_adasyn.append(f1_score(y_test, y_pred_adasyn))
    confusion_matrices_adasyn.append(confusion_matrix(y_test, y_pred_adasyn))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote)} After SMOTE:\n', confusion_matrices_smote[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_adasyn)} After ADASYN:\n', confusion_matrices_adasyn[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE
mean_accuracy_smote = np.mean(accuracy_list_smote)
std_accuracy_smote = np.std(accuracy_list_smote)
mean_precision_smote = np.mean(precision_list_smote)
std_precision_smote = np.std(precision_list_smote)
mean_recall_smote = np.mean(recall_list_smote)
std_recall_smote = np.std(recall_list_smote)
mean_f1_smote = np.mean(f1_list_smote)
std_f1_smote = np.std(f1_list_smote)
mean_entropy_smote = np.mean(entropy_list_smote)
std_entropy_smote = np.std(entropy_list_smote)

# Calculate mean and standard deviation of each metric after ADASYN
mean_accuracy_adasyn = np.mean(accuracy_list_adasyn)
std_accuracy_adasyn = np.std(accuracy_list_adasyn)
mean_precision_adasyn = np.mean(precision_list_adasyn)
std_precision_adasyn = np.std(precision_list_adasyn)
mean_recall_adasyn = np.mean(recall_list_adasyn)
std_recall_adasyn = np.std(recall_list_adasyn)
mean_f1_adasyn = np.mean(f1_list_adasyn)
std_f1_adasyn = np.std(f1_list_adasyn)
mean_entropy_adasyn = np.mean(entropy_list_adasyn)
std_entropy_adasyn = np.std(entropy_list_adasyn)

# Calculate mean confusion matrix before and after SMOTE/ADASYN
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote = np.mean(confusion_matrices_smote, axis=0)
mean_conf_matrix_adasyn = np.mean(confusion_matrices_adasyn, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE
print('--- After SMOTE ---')
print('Mean Accuracy:', mean_accuracy_smote)
print('Accuracy Std Dev:', std_accuracy_smote)
print('Mean Precision:', mean_precision_smote)
print('Precision Std Dev:', std_precision_smote)
print('Mean Recall:', mean_recall_smote)
print('Recall Std Dev:', std_recall_smote)
print('Mean F1-score:', mean_f1_smote)
print('F1-score Std Dev:', std_f1_smote)
print('Mean Entropy:', mean_entropy_smote)
print('Entropy Std Dev:', std_entropy_smote)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote)

# Print results after ADASYN
print('--- After ADASYN ---')
print('Mean Accuracy:', mean_accuracy_adasyn)
print('Accuracy Std Dev:', std_accuracy_adasyn)
print('Mean Precision:', mean_precision_adasyn)
print('Precision Std Dev:', std_precision_adasyn)
print('Mean Recall:', mean_recall_adasyn)
print('Recall Std Dev:', std_recall_adasyn)
print('Mean F1-score:', mean_f1_adasyn)
print('F1-score Std Dev:', std_f1_adasyn)
print('Mean Entropy:', mean_entropy_adasyn)
print('Entropy Std Dev:', std_entropy_adasyn)
print('Mean Confusion Matrix:\n', mean_conf_matrix_adasyn)


Confusion Matrix for Fold 1 Before Oversampling:
 [[79  2]
 [ 3  2]]
Confusion Matrix for Fold 1 After SMOTE:
 [[78  3]
 [ 2  3]]
Confusion Matrix for Fold 1 After ADASYN:
 [[77  4]
 [ 2  3]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 2 After SMOTE:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 2 After ADASYN:
 [[79  2]
 [ 2  3]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[78  3]
 [ 4  1]]
Confusion Matrix for Fold 3 After SMOTE:
 [[77  4]
 [ 2  3]]
Confusion Matrix for Fold 3 After ADASYN:
 [[77  4]
 [ 2  3]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[78  2]
 [ 2  4]]
Confusion Matrix for Fold 4 After SMOTE:
 [[77  3]
 [ 1  5]]
Confusion Matrix for Fold 4 After ADASYN:
 [[78  2]
 [ 1  5]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[78  2]
 [ 1  5]]
Confusion Matrix for Fold 5 After SMOTE:
 [[74  6]
 [ 2  4]]
Confusion Matrix for Fold 5 After ADASYN:
 [[75  5]
 [ 1  5]]
Confusion Matrix for Fold 6 Before Overs

UnderSampling

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.under_sampling import NearMiss, ClusterCentroids, TomekLinks, RandomUnderSampler
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_nearmiss = []
precision_list_nearmiss = []
recall_list_nearmiss = []
f1_list_nearmiss = []
entropy_list_nearmiss = []
confusion_matrices_nearmiss = []

accuracy_list_clustercentroids = []
precision_list_clustercentroids = []
recall_list_clustercentroids = []
f1_list_clustercentroids = []
entropy_list_clustercentroids = []
confusion_matrices_clustercentroids = []

accuracy_list_tomeklinks = []
precision_list_tomeklinks = []
recall_list_tomeklinks = []
f1_list_tomeklinks = []
entropy_list_tomeklinks = []
confusion_matrices_tomeklinks = []

accuracy_list_random = []
precision_list_random = []
recall_list_random = []
f1_list_random = []
entropy_list_random = []
confusion_matrices_random = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model without undersampling
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before undersampling
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before undersampling
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before undersampling
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply NearMiss to the training data
    nearmiss = NearMiss(version=1)
    X_train_nearmiss, y_train_nearmiss = nearmiss.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with NearMiss
    xgb_model_nearmiss = XGBClassifier(random_state=42)
    xgb_model_nearmiss.fit(X_train_nearmiss, y_train_nearmiss)
    
    # Predictions and probabilities after NearMiss
    y_pred_nearmiss = xgb_model_nearmiss.predict(X_test)
    y_prob_nearmiss = xgb_model_nearmiss.predict_proba(X_test)
    
    # Calculate entropy after NearMiss
    entropy_nearmiss = calculate_entropy(y_prob_nearmiss)
    entropy_list_nearmiss.append(entropy_nearmiss)
    
    # Store metrics after NearMiss
    accuracy_list_nearmiss.append(accuracy_score(y_test, y_pred_nearmiss))
    precision_list_nearmiss.append(precision_score(y_test, y_pred_nearmiss))
    recall_list_nearmiss.append(recall_score(y_test, y_pred_nearmiss))
    f1_list_nearmiss.append(f1_score(y_test, y_pred_nearmiss))
    confusion_matrices_nearmiss.append(confusion_matrix(y_test, y_pred_nearmiss))
    
    # Apply ClusterCentroids to the training data
    clustercentroids = ClusterCentroids(random_state=42)
    X_train_clustercentroids, y_train_clustercentroids = clustercentroids.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with ClusterCentroids
    xgb_model_clustercentroids = XGBClassifier(random_state=42)
    xgb_model_clustercentroids.fit(X_train_clustercentroids, y_train_clustercentroids)
    
    # Predictions and probabilities after ClusterCentroids
    y_pred_clustercentroids = xgb_model_clustercentroids.predict(X_test)
    y_prob_clustercentroids = xgb_model_clustercentroids.predict_proba(X_test)
    
    # Calculate entropy after ClusterCentroids
    entropy_clustercentroids = calculate_entropy(y_prob_clustercentroids)
    entropy_list_clustercentroids.append(entropy_clustercentroids)
    
    # Store metrics after ClusterCentroids
    accuracy_list_clustercentroids.append(accuracy_score(y_test, y_pred_clustercentroids))
    precision_list_clustercentroids.append(precision_score(y_test, y_pred_clustercentroids))
    recall_list_clustercentroids.append(recall_score(y_test, y_pred_clustercentroids))
    f1_list_clustercentroids.append(f1_score(y_test, y_pred_clustercentroids))
    confusion_matrices_clustercentroids.append(confusion_matrix(y_test, y_pred_clustercentroids))
    
    # Apply TomekLinks to the training data
    tomeklinks = TomekLinks()
    X_train_tomeklinks, y_train_tomeklinks = tomeklinks.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with TomekLinks
    xgb_model_tomeklinks = XGBClassifier(random_state=42)
    xgb_model_tomeklinks.fit(X_train_tomeklinks, y_train_tomeklinks)
    
    # Predictions and probabilities after TomekLinks
    y_pred_tomeklinks = xgb_model_tomeklinks.predict(X_test)
    y_prob_tomeklinks = xgb_model_tomeklinks.predict_proba(X_test)
    
    # Calculate entropy after TomekLinks
    entropy_tomeklinks = calculate_entropy(y_prob_tomeklinks)
    entropy_list_tomeklinks.append(entropy_tomeklinks)
    
    # Store metrics after TomekLinks
    accuracy_list_tomeklinks.append(accuracy_score(y_test, y_pred_tomeklinks))
    precision_list_tomeklinks.append(precision_score(y_test, y_pred_tomeklinks))
    recall_list_tomeklinks.append(recall_score(y_test, y_pred_tomeklinks))
    f1_list_tomeklinks.append(f1_score(y_test, y_pred_tomeklinks))
    confusion_matrices_tomeklinks.append(confusion_matrix(y_test, y_pred_tomeklinks))
    
    # Apply RandomUnderSampler to the training data
    random_undersampler = RandomUnderSampler(random_state=42)
    X_train_random, y_train_random = random_undersampler.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with RandomUnderSampler
    xgb_model_random = XGBClassifier(random_state=42)
    xgb_model_random.fit(X_train_random, y_train_random)
    
    # Predictions and probabilities after RandomUnderSampler
    y_pred_random = xgb_model_random.predict(X_test)
    y_prob_random = xgb_model_random.predict_proba(X_test)
    
    # Calculate entropy after RandomUnderSampler
    entropy_random = calculate_entropy(y_prob_random)
    entropy_list_random.append(entropy_random)
    
    # Store metrics after RandomUnderSampler
    accuracy_list_random.append(accuracy_score(y_test, y_pred_random))
    precision_list_random.append(precision_score(y_test, y_pred_random))
    recall_list_random.append(recall_score(y_test, y_pred_random))
    f1_list_random.append(f1_score(y_test, y_pred_random))
    confusion_matrices_random.append(confusion_matrix(y_test, y_pred_random))
    
    # Print confusion matrices for each method
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Undersampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_nearmiss)} After NearMiss:\n', confusion_matrices_nearmiss[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_clustercentroids)} After ClusterCentroids:\n', confusion_matrices_clustercentroids[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_tomeklinks)} After TomekLinks:\n', confusion_matrices_tomeklinks[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_random)} After RandomUnderSampler:\n', confusion_matrices_random[-1])

# Calculate mean and standard deviation of each metric before undersampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after NearMiss
mean_accuracy_nearmiss = np.mean(accuracy_list_nearmiss)
std_accuracy_nearmiss = np.std(accuracy_list_nearmiss)
mean_precision_nearmiss = np.mean(precision_list_nearmiss)
std_precision_nearmiss = np.std(precision_list_nearmiss)
mean_recall_nearmiss = np.mean(recall_list_nearmiss)
std_recall_nearmiss = np.std(recall_list_nearmiss)
mean_f1_nearmiss = np.mean(f1_list_nearmiss)
std_f1_nearmiss = np.std(f1_list_nearmiss)
mean_entropy_nearmiss = np.mean(entropy_list_nearmiss)
std_entropy_nearmiss = np.std(entropy_list_nearmiss)

# Calculate mean and standard deviation of each metric after ClusterCentroids
mean_accuracy_clustercentroids = np.mean(accuracy_list_clustercentroids)
std_accuracy_clustercentroids = np.std(accuracy_list_clustercentroids)
mean_precision_clustercentroids = np.mean(precision_list_clustercentroids)
std_precision_clustercentroids = np.std(precision_list_clustercentroids)
mean_recall_clustercentroids = np.mean(recall_list_clustercentroids)
std_recall_clustercentroids = np.std(recall_list_clustercentroids)
mean_f1_clustercentroids = np.mean(f1_list_clustercentroids)
std_f1_clustercentroids = np.std(f1_list_clustercentroids)
mean_entropy_clustercentroids = np.mean(entropy_list_clustercentroids)
std_entropy_clustercentroids = np.std(entropy_list_clustercentroids)

# Calculate mean and standard deviation of each metric after TomekLinks
mean_accuracy_tomeklinks = np.mean(accuracy_list_tomeklinks)
std_accuracy_tomeklinks = np.std(accuracy_list_tomeklinks)
mean_precision_tomeklinks = np.mean(precision_list_tomeklinks)
std_precision_tomeklinks = np.std(precision_list_tomeklinks)
mean_recall_tomeklinks = np.mean(recall_list_tomeklinks)
std_recall_tomeklinks = np.std(recall_list_tomeklinks)
mean_f1_tomeklinks = np.mean(f1_list_tomeklinks)
std_f1_tomeklinks = np.std(f1_list_tomeklinks)
mean_entropy_tomeklinks = np.mean(entropy_list_tomeklinks)
std_entropy_tomeklinks = np.std(entropy_list_tomeklinks)

# Calculate mean and standard deviation of each metric after RandomUnderSampler
mean_accuracy_random = np.mean(accuracy_list_random)
std_accuracy_random = np.std(accuracy_list_random)
mean_precision_random = np.mean(precision_list_random)
std_precision_random = np.std(precision_list_random)
mean_recall_random = np.mean(recall_list_random)
std_recall_random = np.std(recall_list_random)
mean_f1_random = np.mean(f1_list_random)
std_f1_random = np.std(f1_list_random)
mean_entropy_random = np.mean(entropy_list_random)
std_entropy_random = np.std(entropy_list_random)

# Calculate mean confusion matrix before and after each method
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_nearmiss = np.mean(confusion_matrices_nearmiss, axis=0)
mean_conf_matrix_clustercentroids = np.mean(confusion_matrices_clustercentroids, axis=0)
mean_conf_matrix_tomeklinks = np.mean(confusion_matrices_tomeklinks, axis=0)
mean_conf_matrix_random = np.mean(confusion_matrices_random, axis=0)

# Print results before undersampling
print('--- Before Undersampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after NearMiss
print('--- After NearMiss ---')
print('Mean Accuracy:', mean_accuracy_nearmiss)
print('Accuracy Std Dev:', std_accuracy_nearmiss)
print('Mean Precision:', mean_precision_nearmiss)
print('Precision Std Dev:', std_precision_nearmiss)
print('Mean Recall:', mean_recall_nearmiss)
print('Recall Std Dev:', std_recall_nearmiss)
print('Mean F1-score:', mean_f1_nearmiss)
print('F1-score Std Dev:', std_f1_nearmiss)
print('Mean Entropy:', mean_entropy_nearmiss)
print('Entropy Std Dev:', std_entropy_nearmiss)
print('Mean Confusion Matrix:\n', mean_conf_matrix_nearmiss)

# Print results after ClusterCentroids
print('--- After ClusterCentroids ---')
print('Mean Accuracy:', mean_accuracy_clustercentroids)
print('Accuracy Std Dev:', std_accuracy_clustercentroids)
print('Mean Precision:', mean_precision_clustercentroids)
print('Precision Std Dev:', std_precision_clustercentroids)
print('Mean Recall:', mean_recall_clustercentroids)
print('Recall Std Dev:', std_recall_clustercentroids)
print('Mean F1-score:', mean_f1_clustercentroids)
print('F1-score Std Dev:', std_f1_clustercentroids)
print('Mean Entropy:', mean_entropy_clustercentroids)
print('Entropy Std Dev:', std_entropy_clustercentroids)
print('Mean Confusion Matrix:\n', mean_conf_matrix_clustercentroids)

# Print results after TomekLinks
print('--- After TomekLinks ---')
print('Mean Accuracy:', mean_accuracy_tomeklinks)
print('Accuracy Std Dev:', std_accuracy_tomeklinks)
print('Mean Precision:', mean_precision_tomeklinks)
print('Precision Std Dev:', std_precision_tomeklinks)
print('Mean Recall:', mean_recall_tomeklinks)
print('Recall Std Dev:', std_recall_tomeklinks)
print('Mean F1-score:', mean_f1_tomeklinks)
print('F1-score Std Dev:', std_f1_tomeklinks)
print('Mean Entropy:', mean_entropy_tomeklinks)
print('Entropy Std Dev:', std_entropy_tomeklinks)
print('Mean Confusion Matrix:\n', mean_conf_matrix_tomeklinks)

# Print results after RandomUnderSampler
print('--- After RandomUnderSampler ---')
print('Mean Accuracy:', mean_accuracy_random)
print('Accuracy Std Dev:', std_accuracy_random)
print('Mean Precision:', mean_precision_random)
print('Precision Std Dev:', std_precision_random)
print('Mean Recall:', mean_recall_random)
print('Recall Std Dev:', std_recall_random)
print('Mean F1-score:', mean_f1_random)
print('F1-score Std Dev:', std_f1_random)
print('Mean Entropy:', mean_entropy_random)
print('Entropy Std Dev:', std_entropy_random)
print('Mean Confusion Matrix:\n', mean_conf_matrix_random)


Confusion Matrix for Fold 1 Before Undersampling:
 [[79  2]
 [ 3  2]]
Confusion Matrix for Fold 1 After NearMiss:
 [[35 46]
 [ 0  5]]
Confusion Matrix for Fold 1 After ClusterCentroids:
 [[56 25]
 [ 0  5]]
Confusion Matrix for Fold 1 After TomekLinks:
 [[79  2]
 [ 2  3]]
Confusion Matrix for Fold 1 After RandomUnderSampler:
 [[70 11]
 [ 0  5]]
Confusion Matrix for Fold 2 Before Undersampling:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 2 After NearMiss:
 [[38 43]
 [ 0  5]]
Confusion Matrix for Fold 2 After ClusterCentroids:
 [[63 18]
 [ 1  4]]
Confusion Matrix for Fold 2 After TomekLinks:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 2 After RandomUnderSampler:
 [[74  7]
 [ 1  4]]
Confusion Matrix for Fold 3 Before Undersampling:
 [[78  3]
 [ 4  1]]
Confusion Matrix for Fold 3 After NearMiss:
 [[39 42]
 [ 1  4]]
Confusion Matrix for Fold 3 After ClusterCentroids:
 [[68 13]
 [ 1  4]]
Confusion Matrix for Fold 3 After TomekLinks:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 3 After Ra

ThresholdMoving

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Function to find the best threshold for F1-score
def find_best_threshold(y_true, y_prob, thresholds):
    best_f1 = 0
    best_threshold = 0.5
    for threshold in thresholds:
        y_pred = (y_prob[:, 1] >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold, best_f1

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics
best_thresholds = []
f1_list = []
accuracy_list = []
precision_list = []
recall_list = []
entropy_list = []
confusion_matrices = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Get predicted probabilities on training set
    y_prob_train = xgb_model.predict_proba(X_train)
    
    # Define a range of thresholds to test
    thresholds = np.arange(0.1, 0.9, 0.01)
    
    # Find the best threshold based on training set
    best_threshold, _ = find_best_threshold(y_train, y_prob_train, thresholds)
    best_thresholds.append(best_threshold)
    
    # Get predicted probabilities on test set
    y_prob_test = xgb_model.predict_proba(X_test)
    
    # Calculate entropy
    entropy = calculate_entropy(y_prob_test)
    entropy_list.append(entropy)
    
    # Make predictions using the best threshold
    y_pred = (y_prob_test[:, 1] >= best_threshold).astype(int)
    
    # Store the metrics
    f1_list.append(f1_score(y_test, y_pred))
    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred))
    recall_list.append(recall_score(y_test, y_pred))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))
    
    print(f'Best Threshold for Fold {len(best_thresholds)}: {best_threshold}')
    print(f'Confusion Matrix for Fold {len(best_thresholds)}:\n', confusion_matrices[-1])

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)
mean_entropy = np.mean(entropy_list)
std_entropy = np.std(entropy_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0).astype(int)

# Print the results
print('--- Overall Results ---')
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_entropy)
print('Entropy Std Dev:', std_entropy)
print('Mean Confusion Matrix:\n', mean_conf_matrix)

Best Threshold for Fold 1: 0.44999999999999984
Confusion Matrix for Fold 1:
 [[79  2]
 [ 3  2]]
Best Threshold for Fold 2: 0.4299999999999998
Confusion Matrix for Fold 2:
 [[78  3]
 [ 3  2]]
Best Threshold for Fold 3: 0.2599999999999999
Confusion Matrix for Fold 3:
 [[78  3]
 [ 2  3]]
Best Threshold for Fold 4: 0.17999999999999997
Confusion Matrix for Fold 4:
 [[77  3]
 [ 1  5]]
Best Threshold for Fold 5: 0.22999999999999995
Confusion Matrix for Fold 5:
 [[78  2]
 [ 1  5]]
Best Threshold for Fold 6: 0.22999999999999995
Confusion Matrix for Fold 6:
 [[77  3]
 [ 2  4]]
Best Threshold for Fold 7: 0.33999999999999986
Confusion Matrix for Fold 7:
 [[78  2]
 [ 3  3]]
Best Threshold for Fold 8: 0.3599999999999999
Confusion Matrix for Fold 8:
 [[79  1]
 [ 1  5]]
Best Threshold for Fold 9: 0.3599999999999999
Confusion Matrix for Fold 9:
 [[78  2]
 [ 2  3]]
Best Threshold for Fold 10: 0.3599999999999999
Confusion Matrix for Fold 10:
 [[77  3]
 [ 0  5]]
--- Overall Results ---
Mean Accuracy: 0.95

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier

# هزینه‌ها
cost_false_positive = 1  # هزینه مثبت کاذب
cost_false_negative = 100  # هزینه منفی کاذب

# تابع برای محاسبه هزینه کلی
def calculate_total_cost(y_true, y_pred, cost_fp, cost_fn):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    total_cost = (fp * cost_fp) + (fn * cost_fn)
    return total_cost

# تابع برای پیدا کردن بهترین آستانه
def find_best_threshold_cost(y_true, y_prob, thresholds, cost_fp, cost_fn):
    best_f1 = 0
    best_threshold = 0.5
    best_cost = float('inf')
    for threshold in thresholds:
        y_pred = (y_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        total_cost = calculate_total_cost(y_true, y_pred, cost_fp, cost_fn)
        
        # می‌خواهیم بهترین F1-اسکور را با کمترین هزینه پیدا کنیم
        if f1 > best_f1 or (f1 == best_f1 and total_cost < best_cost):
            best_f1 = f1
            best_threshold = threshold
            best_cost = total_cost
            
    return best_threshold, best_f1, best_cost

# تعداد تقسیم‌ها
n_splits = 10

# مقداردهی StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# لیست‌ها برای ذخیره متریک‌ها
best_thresholds = []
train_f1_list = []
test_f1_list = []
costs = []
roc_auc_list = []
recall_list = []
precision_list = []
accuracy_list = []
conf_matrices = []

# حلقه برای تقسیم‌های StratifiedKFold
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # مقداردهی و آموزش مدل XGBoost
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # گرفتن احتمال‌های پیش‌بینی شده برای داده‌های آموزشی
    y_prob_train = xgb_model.predict_proba(X_train)[:, 1]
    
    # تعریف بازه‌ای از آستانه‌ها
    thresholds = np.arange(0.1, 0.9, 0.01)
    
    # پیدا کردن بهترین آستانه بر اساس داده‌های آموزشی
    best_threshold, train_f1, best_cost = find_best_threshold_cost(y_train, y_prob_train, thresholds, cost_false_positive, cost_false_negative)
    best_thresholds.append(best_threshold)
    costs.append(best_cost)
    train_f1_list.append(train_f1)
    
    # پیش‌بینی با استفاده از بهترین آستانه بر روی داده‌های آزمایشی
    y_prob_test = xgb_model.predict_proba(X_test)[:, 1]
    y_pred = (y_prob_test >= best_threshold).astype(int)
    
    # محاسبه متریک‌ها برای داده‌های آزمایشی
    test_f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob_test)
    cm = confusion_matrix(y_test, y_pred)
    
    # ذخیره متریک‌ها
    test_f1_list.append(test_f1)
    recall_list.append(recall)
    precision_list.append(precision)
    accuracy_list.append(accuracy)
    roc_auc_list.append(roc_auc)
    conf_matrices.append(cm)

    # چاپ نتایج برای هر تقسیم
    print(f'Fold {len(best_thresholds)} - Best Threshold: {best_threshold}')
    print(f'Test F1-Score: {test_f1}, Recall: {recall}, Precision: {precision}, Accuracy: {accuracy}')
    print(f'ROC AUC: {roc_auc}, Total Cost: {best_cost}')
    print(f'Confusion Matrix:\n{cm}\n')

# محاسبه میانگین برای هر متریک
mean_test_f1 = np.mean(test_f1_list)
mean_recall = np.mean(recall_list)
mean_precision = np.mean(precision_list)
mean_accuracy = np.mean(accuracy_list)
mean_roc_auc = np.mean(roc_auc_list)
mean_cost = np.mean(costs)

# محاسبه میانگین ماتریس‌های آشفتگی
mean_conf_matrix = np.mean(conf_matrices, axis=0)

# چاپ نتایج کلی
print('--- Overall Results ---')
print('Mean Test F1-score:', mean_test_f1)
print('Mean Recall:', mean_recall)
print('Mean Precision:', mean_precision)
print('Mean Accuracy:', mean_accuracy)
print('Mean ROC AUC:', mean_roc_auc)
print('Mean Total Cost:', mean_cost)
print('Mean Confusion Matrix:\n', mean_conf_matrix)

Fold 1 - Best Threshold: 0.44999999999999984
Test F1-Score: 0.4444444444444444, Recall: 0.4, Precision: 0.5, Accuracy: 0.9418604651162791
ROC AUC: 0.9580246913580247, Total Cost: 0
Confusion Matrix:
[[79  2]
 [ 3  2]]

Fold 2 - Best Threshold: 0.4299999999999998
Test F1-Score: 0.4, Recall: 0.4, Precision: 0.4, Accuracy: 0.9302325581395349
ROC AUC: 0.888888888888889, Total Cost: 0
Confusion Matrix:
[[78  3]
 [ 3  2]]

Fold 3 - Best Threshold: 0.2599999999999999
Test F1-Score: 0.5454545454545454, Recall: 0.6, Precision: 0.5, Accuracy: 0.9418604651162791
ROC AUC: 0.9160493827160494, Total Cost: 0
Confusion Matrix:
[[78  3]
 [ 2  3]]

Fold 4 - Best Threshold: 0.17999999999999997
Test F1-Score: 0.7142857142857143, Recall: 0.8333333333333334, Precision: 0.625, Accuracy: 0.9534883720930233
ROC AUC: 0.96875, Total Cost: 0
Confusion Matrix:
[[77  3]
 [ 1  5]]

Fold 5 - Best Threshold: 0.22999999999999995
Test F1-Score: 0.7692307692307693, Recall: 0.8333333333333334, Precision: 0.714285714285714

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.combine import SMOTETomek  # Import SMOTE + Tomek Links
import numpy as np
import xgboost as xgb  # Import XGBoost

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote_tomek = []
precision_list_smote_tomek = []
recall_list_smote_tomek = []
f1_list_smote_tomek = []
entropy_list_smote_tomek = []
confusion_matrices_smote_tomek = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model without SMOTE + Tomek Links
    xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE + Tomek Links
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE + Tomek Links
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE + Tomek Links
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE + Tomek Links to the training data
    smote_tomek = SMOTETomek(random_state=42)
    X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with SMOTE + Tomek Links
    xgb_model_smote_tomek = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model_smote_tomek.fit(X_train_smote_tomek, y_train_smote_tomek)
    
    # Predictions and probabilities after SMOTE + Tomek Links
    y_pred_smote_tomek = xgb_model_smote_tomek.predict(X_test)
    y_prob_smote_tomek = xgb_model_smote_tomek.predict_proba(X_test)
    
    # Calculate entropy after SMOTE + Tomek Links
    entropy_smote_tomek = calculate_entropy(y_prob_smote_tomek)
    entropy_list_smote_tomek.append(entropy_smote_tomek)
    
    # Store metrics after SMOTE + Tomek Links
    accuracy_list_smote_tomek.append(accuracy_score(y_test, y_pred_smote_tomek))
    precision_list_smote_tomek.append(precision_score(y_test, y_pred_smote_tomek))
    recall_list_smote_tomek.append(recall_score(y_test, y_pred_smote_tomek))
    f1_list_smote_tomek.append(f1_score(y_test, y_pred_smote_tomek))
    confusion_matrices_smote_tomek.append(confusion_matrix(y_test, y_pred_smote_tomek))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote_tomek)} After SMOTE + Tomek Links:\n', confusion_matrices_smote_tomek[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE + Tomek Links
mean_accuracy_smote_tomek = np.mean(accuracy_list_smote_tomek)
std_accuracy_smote_tomek = np.std(accuracy_list_smote_tomek)
mean_precision_smote_tomek = np.mean(precision_list_smote_tomek)
std_precision_smote_tomek = np.std(precision_list_smote_tomek)
mean_recall_smote_tomek = np.mean(recall_list_smote_tomek)
std_recall_smote_tomek = np.std(recall_list_smote_tomek)
mean_f1_smote_tomek = np.mean(f1_list_smote_tomek)
std_f1_smote_tomek = np.std(f1_list_smote_tomek)
mean_entropy_smote_tomek = np.mean(entropy_list_smote_tomek)
std_entropy_smote_tomek = np.std(entropy_list_smote_tomek)

# Calculate mean confusion matrix before and after SMOTE + Tomek Links
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote_tomek = np.mean(confusion_matrices_smote_tomek, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE + Tomek Links
print('--- After SMOTE + Tomek Links ---')
print('Mean Accuracy:', mean_accuracy_smote_tomek)
print('Accuracy Std Dev:', std_accuracy_smote_tomek)
print('Mean Precision:', mean_precision_smote_tomek)
print('Precision Std Dev:', std_precision_smote_tomek)
print('Mean Recall:', mean_recall_smote_tomek)
print('Recall Std Dev:', std_recall_smote_tomek)
print('Mean F1-score:', mean_f1_smote_tomek)
print('F1-score Std Dev:', std_f1_smote_tomek)
print('Mean Entropy:', mean_entropy_smote_tomek)
print('Entropy Std Dev:', std_entropy_smote_tomek)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote_tomek)


Confusion Matrix for Fold 1 Before Oversampling:
 [[79  2]
 [ 3  2]]
Confusion Matrix for Fold 1 After SMOTE + Tomek Links:
 [[77  4]
 [ 2  3]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[78  3]
 [ 3  2]]
Confusion Matrix for Fold 2 After SMOTE + Tomek Links:
 [[76  5]
 [ 2  3]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[78  3]
 [ 4  1]]
Confusion Matrix for Fold 3 After SMOTE + Tomek Links:
 [[78  3]
 [ 2  3]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[78  2]
 [ 2  4]]
Confusion Matrix for Fold 4 After SMOTE + Tomek Links:
 [[77  3]
 [ 1  5]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[78  2]
 [ 1  5]]
Confusion Matrix for Fold 5 After SMOTE + Tomek Links:
 [[75  5]
 [ 1  5]]
Confusion Matrix for Fold 6 Before Oversampling:
 [[79  1]
 [ 3  3]]
Confusion Matrix for Fold 6 After SMOTE + Tomek Links:
 [[77  3]
 [ 1  5]]
Confusion Matrix for Fold 7 Before Oversampling:
 [[78  2]
 [ 5  1]]
Confusion Matrix for Fold 7 After SMOTE + Tomek Links:
 [[76  4]
 [